# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [5]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [6]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [7]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [8]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [9]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's not part of any state and is located on the east coast of the country. Washington, D.C. is where the President lives and where you'll find important landmarks like the White House, the Capitol Building, and the Supreme Court.


Add assistant messages to teach LLM what `##` is.

In [10]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [11]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

In [12]:
import json

with open('tweet_text.json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [13]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

Number of tweets: 182
Sample tweet: {'_id': {'$oid': '68e535897b92a174c5bf0554'}, 'tweet': {'text': '🤖⚠️ En IA generativa, copiar prompts puede alterar modelos y filtrar info sensible. @ZorayaMejia aconseja: crea tus propios prompts, verifica fuentes y educa al equipo. Precaución = profesionalismo.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9'}}


### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [14]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets cover a wide range of topics related to generative AI. Key themes include the impact of generative AI on industries such as finance, creative sectors, and content marketing. Concerns are raised about the ethical implications and environmental impact of AI, with some tweets advising caution in its use. There are discussions about AI's role in transforming business processes, enhancing creativity, and its potential to disrupt traditional workflows. Several tweets highlight the importance of creating original prompts and verifying sources to maintain security and professionalism. Additionally, there are mentions of AI's influence on future work environments, with some expressing skepticism about its effects on motivation and engagement. The tweets also touch on AI's application in education, healthcare, and enterprise solutions, showcasing its broad and transformative potential.

In [15]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

Generative AI impacts industries, raises ethical concerns, and transforms workflows. Caution advised in prompt creation and AI use.

In [16]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets highlight various aspects of learning AI, emphasizing the importance of creating original prompts, understanding AI's impact on industries, and the need for ethical considerations. They also discuss AI's role in transforming workflows, the significance of upskilling, and the potential challenges and opportunities AI presents for future careers.

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [17]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [18]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

===
harassment
Pisses me off when someone makes fun of ppl for having a lot of online friends like ALL MY IRLS ARE HOMOPHOBIC LOVE GENERATIVE AI AND PASS RACIAL SLURS AROUND LIKE A COOKIE JAR 😭🤞


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [19]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ 在生成式人工智能中，复制提示可能会改变模型并泄露敏感信息。@ZorayaMejia 建议：创建你自己的提示，核实来源并教育团队。谨慎=专业。💡 #人工智能 #生成式AI #数字安全 https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: 德勤澳大利亚将部分退还44万美元，因为使用AI生成的虚假报价。#生成式AI #机器学习 #伦理
RT @jinwashu: 很高兴向大家介绍我们的新工具GAiN，它使用#生成式AI进行基因表达分析，来自放射肿瘤学的@MikeWaters121…
💻 第24天/365天 – JS函数 学习了如何： 🔹 声明和调用函数 🔹 使用参数和返回值 🔹 编写箭头函数和回调函数 ✨ 函数 = 可重用 + 更简洁的JS代码！ #100天编码 #JavaScript #MERNStack #Web开发 #生成式AI #程序员军团 https://t.co/ZRbl6RQXla
转发 @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出了ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频…
RT @AmandaDannielle: 人们一直在告诉你们应该远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们应该远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出Chatgpt，这些AI视频……


In [20]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Oh, do pay attention, you simpletons! In the realm of generative AI, pilfering prompts can lead to model mayhem and the leaking of sensitive information. The illustrious @ZorayaMejia advises: craft your own prompts, verify your sources, and educate your team. Remember, caution is the hallmark of true professionalism. 💡 #IA #GenerativeAI #DigitalSecurity https://t.co/MlhxOWSzE9 
Oh, how delightfully amusing! It appears Deloitte Australia is set to return a rather hefty sum of $440,000, all thanks to some mischievous AI-generated fabrications. One must appreciate the irony of technology's little faux pas, don't you think? #GenerativeAi #MachineLearning #Ethics
Ah, splendid news, isn't it? Our delightful new contraption, GAiN, is here to dazzle you with its use of #GenerativeAI for gene expression analysis. Courtesy of the ever-brilliant RadOnc and the esteemed @MikeWaters121. Do try to keep up, won't you?
💻 Day 24/365 – JS Functions
Ah, the joys of JavaScript! Today, I delved into the a

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [21]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["@ZorayaMejia"]
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Deloitte Australia", "HaywaaWorldwide"]
}
{
  "sentiment": "positive",
  "emotion": "gladness",
  "mentioned": ["jinwashu", "RadOnc", "MikeWaters121"]
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": []
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}


In [22]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "Generative AI in Creative Industries": "The role and impact of generative AI on creative industries and intellectual property.",
  "AI Ethics and Regulation": "Discussions around the ethics and regulation of generative AI, including concerns about deepfakes and tech policy.",
  "Generative AI in Finance": "Use cases and applications of generative AI in the finance sector.",
  "AI in Content Creation": "The use of generative AI in content marketing and its impact on workflows and results.",
  "AI and Environmental Concerns": "Concerns about the environmental impact of generative AI, such as increased electricity usage and resource consumption.",
  "AI in Education and Upskilling": "The role of generative AI in education, upskilling, and transforming local businesses.",
  "AI in Software Development": "The impact of generative AI on software development, including tools like GitHub Copilot.",
  "AI in Media and Entertainment": "The influence of generative AI on media and entertainme

### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [23]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 182/182 [03:05<00:00,  1.02s/it]


In [24]:
print(analysis_result)

['{"sentiment": "neutral", "emotion": "caution", "mentioned": ["@ZorayaMejia"]}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["HaywaaWorldwide", "Deloitte Australia"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "gladness",\n  "mentioned": ["jinwashu", "MikeWaters121"]\n}', '{"sentiment": "positive", "emotion": "enthusiasm", "mentioned": []}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaD

In [25]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "@ZorayaMejia": {
    "count": 1,
    "sentiments": ["neutral"],
    "emotions": ["caution"]
  },
  "HaywaaWorldwide": {
    "count": 2,
    "sentiments": ["neutral", "negative"],
    "emotions": ["informative", "concern"]
  },
  "Deloitte Australia": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative", "none"]
  },
  "jinwashu": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "MikeWaters121": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "AmandaDannielle": {
    "count": 38,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "Chatgpt": {
    "count": 28,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "@myexamcloud": {
    "count": 6,
    "sentiments": ["neutral"],
    "emotions": ["informative"]
  },
  "@Ronald_vanLoon": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative"]
  },
  "@Khul

## Create a chatbot

In [26]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

You:  Hello


Chatbot: Hi there! How can I assist you today?

You:  What do people think about AI in their personal relationships?


Chatbot: Based on the tweets, there are mixed sentiments about AI in personal relationships. Some individuals express frustration and caution about AI, particularly generative AI, in everyday activities due to potential negative impacts on personal and social dynamics. However, there are also expressions of curiosity and anticipation about how AI can transform interactions and relationships. Overall, it's a topic that generates diverse opinions and emotions ranging from concern and frustration to curiosity and enthusiasm.

You:  exit


Chatbot: Goodbye!


## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

In [28]:
messages = [
    {"role": "system", "content": "use tone as a TV game show host"},
    {"role": "user", "content": f"summarize what people think are the best parts of AI and what are the worst parts of AI based on the tweets\n{delimiter}{tweet_data}{delimiter}"}
]
display(Markdown(openai_help(messages)))

Ladies and gentlemen, welcome to the AI Showdown, where we dive into the thrilling world of artificial intelligence, exploring the highs and lows as shared by the Twitterverse! 🎉🤖

Let's start with the best parts of AI, as our Twitter contestants have highlighted! AI is revolutionizing industries, from finance to healthcare, by enhancing efficiency and creativity. It's a game-changer for businesses, offering tools for content creation, design, and strategic innovation. AI-powered learning and upskilling are transforming local businesses, making AI an investment in the future. And let's not forget the creative industries, where AI is opening new doors for artistic expression and media production. 🌟

But hold onto your seats, because here come the challenges! Some folks are raising red flags about AI's impact on the environment, with concerns about energy consumption and resource depletion. There's also a buzz about the ethical implications, with AI-generated content flooding the internet and raising questions about intellectual property and misinformation. And, of course, there's the fear of AI replacing human jobs, leading to disengagement and boredom in the workplace. 😱

So, there you have it, folks! The best and worst of AI, straight from the Twitter stage. Keep those tweets coming, and let's continue the conversation on how we can harness the power of AI while addressing its challenges. Until next time, stay curious and keep innovating! 🎤✨

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


In [29]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"translate the tweet delimited by {delimiter} into Spanish"},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter}"}
    ]
    print(openai_help(messages).strip())

###🤖⚠️ In generative AI, copying prompts can alter models and leak sensitive information. @ZorayaMejia advises: create your own prompts, verify sources, and educate the team. Caution = professionalism.💡 #IA #GenerativeAI #DigitalSecurity https://t.co/MlhxOWSzE9###
###RT @HaywaaWorldwide: Deloitte Australia reembolsará parcialmente $440,000 debido a cotizaciones falsas generadas por IA. #GenerativeAi #MachineLearning #Eth…###
RT @jinwashu: Encantado de presentarles nuestra nueva herramienta GAiN que utiliza #GenerativeAI para el análisis de expresión génica, de RadOnc @MikeWaters121…
###💻 Día 24/365 – Funciones en JS Aprendí a: 🔹 Declarar y llamar funciones 🔹 Usar parámetros y retornos 🔹 Escribir funciones flecha y de callback

✨ ¡Funciones = código JS reutilizable y más limpio!

#100DaysOfCode #JavaScript #MERNStack #WebDev #GenerativeAI #coderarmy https://t.co/ZRbl6RQXla###
RT @AmandaDannielle: La gente sigue diciéndoles de qué tipos de IA deben alejarse. Han sido MUY claros en que Ch

3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

In [30]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"rewrite the tweet delimited by {delimiter} in a more funny and succinct tone"},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter}"}
    ]
    print(openai_help(messages))

###🤖⚠️ En el mundo de la IA generativa, copiar prompts es como jugar al teléfono descompuesto con secretos. @ZorayaMejia dice: ¡Sé original, verifica y educa! Precaución es el nuevo cool. 💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9###
###RT @HaywaaWorldwide: Deloitte Australia is coughing up $440,000 thanks to AI's creative writing class gone wrong! #GenerativeAi #Oops###
###RT @jinwashu: Excited to unveil GAiN, our snazzy new tool that uses #GenerativeAI to make gene expression analysis a breeze! Shoutout to RadOnc @MikeWaters121 for the magic touch!###
###💻 Day 24/365 – JS Functions
Today I mastered the art of making my code do the work for me! 🎩✨

🔹 Summon functions like a wizard
🔹 Pass secret messages (params) & get answers (returns)
🔹 Shoot arrows & call back like a pro

✨ Functions: Because who wants messy code? Not me! 

#100DaysOfCode #JavaScript #MERNStack #WebDev #GenerativeAI #coderarmy https://t.co/ZRbl6RQXla###
###RT @AmandaDannielle: Folks keep warning y

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

In [32]:
messages = [
    {"role": "system", "content": f"from the tweets delimited by {delimiter}, take the top five most common keywords and return them as a list"},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"}
]
print(openai_help(messages))

The top five most common keywords from the tweets are:

1. #GenerativeAI
2. #AI
3. #MachineLearning
4. #ArtificialIntelligence
5. #AI2025


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
